Questions :
* On doit toujours considérer la ruine?
* Possibilité de coder en C++ ?
* Pour la question 3, c'est quoi la loi de $\xi_n$?

**Tester le code avec des événements plus probable!!!**
## To do list:
* Q1-1
#####* Splitting (Diane) (faudrait jouer avec le nombre de niveaux de splitting, donc avec la valeur de la variable "seuil", pour voir ce qui donne la meilleure précision)
#####* Echantillonage d'importance (Peng-Wei)
#####* Chaîne de Markov -> Regarder l'article FodraPhamMicrostructure2015
#####* Intervalle de confiance pour la Monte-Carlo simple (et pour les autres ?)

* Q1-2
#####* Quantile processus AR(1) (comme dans calcul liste_a Q1-1) (Diane)
#####* Quantile échantillonnage d'importance (Diane)
* Q2
#####* La même chose mais il faut prendre en compte les différents instants de saut. Donc ça change tout.
#####* M-C (Peng-Wei

* Q
* Q3
# Finished :
* Q1
#####* Monte-Carlo naïve


In [1]:
import numpy as np
import math
import matplotlib.pyplot

## Q1

Dans cette première modélisation simplifiée, on considère donc que le prix $P_t$ est un processus de Poisson de paramètres $\lambda, \nu$ où $\nu$ est la loi des incréments $J_n$. 

Pour un temps d'attente moyen entre deux sauts de $300s$, on prend $\lambda = \dfrac{1}{300}$


In [2]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#### On définit nu, la loi des incréments
# Ancien : plus lent
saut_1_ancien = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2_ancien = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3

# Nouveau : beaucoup plus vite
value_1 = np.array([-1, 1])
value_2 = np.array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
saut_1 = lambda x : value_1[np.random.randint(low=2, size=x)]
saut_2 = lambda x : value_2[np.random.randint(low=12, size=x)]

# Les paramètres
P0 = 35
T = 4*60*60
lamb = 1/300

###  Q1 - 2 Quantile


Pour les intervalles de confiance des quantiles, voir cours 2 p.3 (y a un TCL pour le quantile)

__Aussi, parle des approximations gaussiennes des poissons composés p6__

D'après le cours:

Si la loi des sauts est centrée et de variance $\sigma^2$
Si on note $N_T$ le nombre de sauts dans $[0,T]$,

$$N \sim \mathcal{P}(\lambda T)$$
Alors le prix à $T$, $X_T$,  converge en loi vers $\mathcal{N}(P_0,\lambda T \sigma^2)$ quand $\lambda T \rightarrow + \infty$

Ici on a $\lambda T = 48$

Ca serait intéressant de comparer les quantiles obtenus en Q1.2 avec les quantiles de la loi normale $\mathcal{N}(P_0,\lambda T \sigma^2)$, où $\sigma^2=1$ si $m=1$ et $\sigma^2 = ...$ si $m=3$ (normalement, devraient être proches)

In [3]:
#Monte-Carlo accéléré - Méthode 2

# Poisson composé naïf
def Poisson_compose(P0, T, lamb, saut, size):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = np.cumsum(N)                  # La valeur N[i] - N[i - 1] est égale à N pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] + 1 et N[i + 1] suit la loi voulue
  J = saut(N[-1] + 1)               # Sauts
  J = np.cumsum(J)
  M = J[N[1:]] - J[N[:-1]]          # On veut la somme entre l'indice N[i] + 1 et N[i + 1]
  return P0 + M # échantillon de taille (size)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question1_2_naive(size, quant, saut, P0, T, lamb):
  return quantile(Poisson_compose(P0, T, lamb, saut, size), size, quant)

In [4]:
%%time
size = int(1e7)
quant = 1e-4
print(Question1_2_naive(size, quant, saut_1, P0, T, lamb))

#Donne 9 et 61 pour P0 = 35 et m=1 et niveau = 10^-4 
#Donne 5 et 65 pour P0 = 35 et m=1 et niveau = 10^-5 
#Donne 3 et 68 pour P0 = 35 et m=1 et niveau = 10^-6
#Donne -13 et 83 pour P0 = 35 et m=2 et niveau = 10^-4 
#Donne -21 et 92 pour P0 = 35 et m=2 et niveau = 10^-5
#Donne -25 et 94 pour P0 = 35 et m=2 et niveau = 10^-6

(9, 61)
CPU times: user 3.56 s, sys: 1.14 s, total: 4.7 s
Wall time: 4.71 s


In [0]:
#MCMC (chaînes de Markov) - Méthode 3

def Quantile_ergodique(seuil,n,lamb,T,p,P0,saut): #renvoie le quantile de niveau "seuil" du processus de poisson composé

    liste_prix_final = np.zeros(n)

    liste_sauts = liste_sts(lamb,T,saut) 

    for l in range(n):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        liste_sauts = tri_temps(new_liste_sauts)

        liste_prix_final[l] = np.cumsum(liste_sauts[1,:])[-1] +P0

    liste_prix_final.sort()
  
    quantile = liste_prix_final[int(np.ceil(seuil*n))-1]    

  
    return quantile

In [0]:
4.310−3

def Quantile_naif(seuil,n,lamb,T,P0,saut):

    liste_P_t = np.zeros(n)
    for i in range(n):
        N = np.random.poisson(lamb*T)
        increments = saut(N)
        
        liste_P_t[i] = np.sum(increments)+P0
    liste_P_t = np.sort(liste_P_t)
    return liste_P_t[int(np.ceil(n*seuil))-1]

In [0]:
P0 = 35
T = 4*60*60 #conversion en secondes
lamb =  1/300 
n = int(1e6) 

niveau = 1e-5
p=0.5

quantileGauche = Quantile_naif(niveau,n,lamb,T,P0,saut_2) #mettre Quantile_ergodique(niveau,n,lamb,T,p,P0,saut) ou Quantile_naif(niveau,n,lamb,T,P0,saut)
quantileDroit = Quantile_naif(1-niveau,n,lamb,T,P0,saut_2)
print("Quantile de niveau {} : {}".format(niveau,quantileGauche))
print("Quantile de niveau {} : {}".format(1-niveau,quantileDroit))

#Résultats pour Quantile_ergodique:
#Donne 9 et 61 pour P0 = 35 et k=0 et niveau = 10^-4 (c'est à peu près sym/P0 ce qui est rassurant)
#Donne 5 et 66 pour P0 = 35 et k=0 et niveau = 10^-5
#Donne 5 et 63 pour P0 = 35 et k=0 et niveau = 10^-6 (problème... faut augmenter n?) n : 10^5 -> 10^6 : on trouve 2 et 68 ; OK :D

#-15 et 83 pour P0=35 et k=1 et niveau = 10^-4 (sauts plus grands dc on va plus loin, logique; sym/0, logique)
#-16 et 90 pour P0=35 et k=1 et niveau = 10^-5
#Donne -25 et 97 pour P0 = 35 et k=1 et niveau = 10^-6 (en augmentant n de 10^5 à 10^6)

#Résultats Quantile_naïf (plus lent) (n=10^5):
#9 et 62
#5 et 66
#3 et 67
#-13 et 84
#-22 et 87 si on garde n=10^5, -20 et 89 si on passe à n=10^6
# -22 et 97(on passe à n=10^6)

Quantile de niveau 1e-05 : -20.0
Quantile de niveau 0.99999 : 89.0


In [0]:
#Calcul quantile par échantillonnage d'importance
#Voir slides amphi 2 p.12 pour calcul quantile avec changement de probabilité
#Ensuite voir amphi 3 pour changements de proba Poisson composés
#Différence importance sampling/échantillonage d'importance et changement de probabilité ?
#Transfo d'Esscher